In [41]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [42]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
# YOUR CODE HERE!
data = Path('./Resources/lending_data.csv')
df = pd.read_csv(data)

# Review the DataFrame
# YOUR CODE HERE!
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [43]:
# Separate the data into labels and features

# Separate the y variable, the labels
# YOUR CODE HERE!]
y = df["loan_status"]
# Separate the X variable, the features
# YOUR CODE HERE!
X = df.drop(columns="loan_status")

In [44]:
# Review the y variable Series
# YOUR CODE HERE!
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [45]:
# Review the X variable DataFrame
# YOUR CODE HERE!
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [46]:
# Check the balance of our target values
# YOUR CODE HERE!
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [47]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
# YOUR CODE HERE!

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(58152, 7)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [48]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
# YOUR CODE HERE!

classifier = LogisticRegression(random_state=1)

# Fit the model using training data
# YOUR CODE HERE!

classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [49]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9914878250103177
Testing Data Score: 0.9924164259182832


### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [50]:
# Make a prediction using the testing data
# YOUR CODE HERE!

predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [51]:
# Print the balanced_accuracy score of the model
# YOUR CODE HERE!

from sklearn.metrics import accuracy_score
# Display the accuracy score for the test dataset.
accuracy_score(y_test, predictions)

0.9924164259182832

In [52]:
# Generate a confusion matrix for the model
# YOUR CODE HERE!

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, predictions)


array([[18679,    80],
       [   67,   558]], dtype=int64)

In [53]:
# Print the classification report for the model
# YOUR CODE HERE!

from sklearn.metrics import classification_report
target_names = ["0", "1"]
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** WRITE YOUR ANSWER HERE!

The precision/recall/FI score for 0 (healthy loans) shows very good performance for predictions. High risk labels however show slightly lower performance. Very good overall at 0.88 but it shows that not all high risk loans were caught. In the context of credit risk assessement, this performance should be improved.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [54]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model

X_resampled, y_resampled = random_oversampler.fit_resample(X_train, y_train)

In [55]:
# Count the distinct values of the resampled labels data
distinct_values = np.unique(y_resampled)
count = len(distinct_values)

print("Number of distinct values:", count)

Number of distinct values: 2


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [56]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
lr_model = logistic_regression_model.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
# YOUR CODE HERE!
predictions=logistic_regression_model.predict(X_resampled)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [57]:
# Print the balanced_accuracy score of the model 
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy = balanced_accuracy_score(y_resampled, predictions)
print(balanced_accuracy)

0.994180571103648


In [58]:
# Generate a confusion matrix for the model
training_matrix = confusion_matrix(y_resampled, predictions)
print(training_matrix)

[[55945   332]
 [  323 55954]]


In [59]:
# Print the classification report for the model
# Create and save the training classification report
training_report = classification_report(y_resampled, predictions)

# Print the training classification report
print(training_report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     56277
           1       0.99      0.99      0.99     56277

    accuracy                           0.99    112554
   macro avg       0.99      0.99      0.99    112554
weighted avg       0.99      0.99      0.99    112554



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** YOUR ANSWER HERE!

With oversampling, the 0 (healthy loans) decreases minimally (0.01 in all instances), but the number of high risk loan labels increased significantly. This is overall a very good model now. 